# Load Libraries

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
import pandas as pd
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
%matplotlib inline
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import ast
import pickle

/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load Datasets

In [16]:
# Loading the first dataset
X = pd.read_csv('normalized_generic.csv',sep='\t')
X = X.drop('Unnamed: 0',axis=1)


# Merging the two datasets
a=X.loc[X['rumor'] == 0.0]
b=X.loc[X['rumor'] == 1.0]
a=a[0:len(b)]
X=pd.concat([a,b])
X = shuffle(X)
X= X.reset_index(drop=True)

# Defining the last column of the dataframe as the output column of the model
y=X['rumor']

X = X.iloc[:, :-1]

In [33]:
metacontent_features= X.iloc[:,:8]
metacontent_features['rumor']=X.rumor
metacontent_features.to_csv('metacontent_features.csv', sep='\t')
user_features= X.iloc[:,8:32]
user_features['rumor']=X.rumor
user_features.to_csv('user_features.csv', sep='\t')
linguistic_features= X.iloc[:,32:]
linguistic_features.to_csv('linguistic_features.csv', sep='\t')
user_features

/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,profile_age,profile_description,geolocation,opinion_shaper,extended_profile,user_location,statuses_count,user_timezone,verified_account,text_translator,...,gender,male,statuses_per_day,lists_per_day,followers,following,followers_per_day,following_per_day,followers_following_ratio,rumor
0,0.980497,1.0,0.0,0.0,0.0,1.0,0.000272,0.0,0.0,0.0,...,1.0,1.0,2.253168e-07,6.749242e-09,4.954027e-07,0.000020,4.333194e-09,7.989138e-08,1.312275e-07,1.0
1,0.598722,1.0,0.0,0.0,0.0,1.0,0.005578,0.0,0.0,0.0,...,0.0,1.0,7.555898e-06,5.526448e-08,6.921483e-06,0.000501,9.914482e-08,3.270289e-06,7.685058e-08,1.0
2,0.217005,1.0,1.0,0.0,1.0,1.0,0.006534,0.0,0.0,0.0,...,1.0,1.0,2.441644e-05,7.623810e-08,8.888940e-06,0.001512,3.512989e-07,2.722314e-05,3.274603e-08,0.0
3,0.666258,1.0,0.0,0.0,0.0,1.0,0.014427,0.0,0.0,0.0,...,0.0,1.0,1.756040e-05,4.966257e-09,4.670940e-07,0.000002,6.012543e-09,1.370578e-08,8.661013e-07,0.0
4,0.395630,1.0,0.0,0.0,0.0,1.0,0.000721,0.0,0.0,0.0,...,0.0,1.0,1.478817e-06,8.363397e-09,1.643321e-05,0.001688,3.562296e-07,1.666105e-05,5.425107e-08,0.0
5,0.123694,1.0,1.0,0.0,0.0,1.0,0.014444,0.0,0.0,0.0,...,1.0,1.0,9.469499e-05,8.560012e-07,1.967456e-05,0.000000,1.364125e-06,2.772056e-08,1.094437e-04,1.0
6,0.961608,1.0,0.0,0.0,0.0,1.0,0.001484,0.0,0.0,0.0,...,0.0,1.0,1.251702e-06,7.570001e-08,4.571859e-06,0.000216,4.077470e-08,8.790748e-07,1.171976e-07,0.0
7,0.283047,1.0,0.0,0.0,0.0,1.0,0.024170,0.0,0.0,0.0,...,0.0,1.0,6.925049e-05,1.309274e-06,9.498993e-05,0.000205,2.878159e-06,2.843834e-06,2.565051e-06,0.0
8,0.386775,1.0,0.0,0.0,0.0,1.0,0.038195,0.0,0.0,0.0,...,0.0,1.0,8.008412e-05,1.284939e-05,3.859045e-04,0.024103,8.556918e-06,2.433204e-04,8.925126e-08,0.0
9,0.513872,1.0,1.0,0.0,1.0,1.0,0.016308,0.0,0.0,0.0,...,1.0,1.0,2.573636e-05,9.014566e-08,6.327000e-06,0.000787,1.055940e-07,5.981043e-06,4.477762e-08,0.0


In [17]:
a=X.loc[X['rumor'] == 0.0]
b=X.loc[X['rumor'] == 1.0]
a=a[0:len(b)]
X=pd.concat([a,b])
X = shuffle(X)
X= X.reset_index(drop=True)

# Rumor Detector

## Desicion Tree Experimentation

In [5]:
# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [ 2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [20]:
dtree = SVC(C=10, gamma=0.3, kernel='linear',random_state=42)
dtree.fit(X,y)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.3, kernel='linear',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [59]:
from sklearn import tree
import pydot

In [6]:
dt = DecisionTreeClassifier(random_state = 42)
 
# Define multiple metrics to be stored during the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation
dt_random_truth = GridSearchCV(estimator=dt, param_grid=random_grid, cv=5, scoring= scoring, 
                               pre_dispatch=8, refit='accuracy')

# Fit the random search model
dt_random_truth.fit(X,y)

KeyboardInterrupt: 

### Saving the results

In [13]:
output_truth_dt = open('dt_truth_29_08.pkl', 'wb')

pickle.dump(dt_random_truth.cv_results_, output_truth_dt)

output_truth_dt.close()

## Random Forest Experimentation

In [8]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 40)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [ 2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [38]:
# First create the base model to tune
rf = RandomForestClassifier(random_state = 42)

# Define multiple metrics to be stored during the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation 
rf_random_truth = GridSearchCV(estimator=rf, param_grid=random_grid, cv=5, scoring= scoring, 
                               pre_dispatch=8, refit='accuracy')

# Fit the random search model
rf_random_truth.fit(X,y)

### Saving the results

In [14]:
output_truth_rf = open('rf_truth_29_08.pkl', 'wb')

pickle.dump(rf_random_truth.cv_results_, output_truth_rf)

output_truth_rf.close()

## SVM Experimentation

In [4]:
# Exploring the values of C in logarithmic progression
Cs = [0.001, 0.01, 0.1, 1, 10]

# Exploring the values of Gamma in logarithmic progression
gammas = [0.0001, 0.001, 0.01, 0.1, 1]

# Exploring the most suitable kernel
kernel = ['rbf', 'linear']

param_grid = {'C': Cs, 'gamma' : gammas, 'kernel': kernel}

In [ ]:
# Defining the model
svm = SVC(random_state = 42)

# Define multiple performance metrics to be stored in the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation 
grid_search_truth_svm = GridSearchCV(estimator=svm, param_grid=random_grid, cv=5, scoring= scoring, 
                                     pre_dispatch=8, refit='accuracy')

# Fit the model
grid_search_truth_svm.fit(X,y)

### Saving the results

In [ ]:
output_truth_svm = open('rf_truth_29_08.pkl', 'wb')

pickle.dump(grid_search_truth_svm.cv_results_, output_truth_svm)

output_truth_svm.close()

# Humanitarian Relevancy Estimator

## Loading Datasets

In [102]:
X1 = pd.read_csv('generic_text_all.csv',sep='\t', header=None, index_col=False, lineterminator='\n')
X1 = X1.drop(0,axis=1)
X1 = X1.drop(0)
X1 = X1.reset_index(drop=True)
X1['crisis'] = 0.0 

X2 = pd.read_csv('humanitarian_text.csv',sep='\t', header=None, index_col=False, lineterminator='\n')
X2 = X2.drop(0,axis=1)
X2 = X2.reset_index(drop=True)
X2['crisis'] = 1.0

X=pd.concat([X1, X2])
X = shuffle(X)
X= X.reset_index(drop=True)

y = X['crisis']

In [ ]:
a=a.reset_index(drop=True)
for i in range(len(a)):
    
    a[1][i]=ast.literal_eval(a[1][i])

a

In [94]:
def text_process(mess):
    
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
text=[]
    
for i in range(len(K)):
    
    t= ','.join(map(str, K[1][i]))
        
    text.append(t)
    
    
K['text']=text

## Creating a Unigram and a Bigram

In [97]:
bow_transformer = CountVectorizer(analyzer=text_process, ngram_range=(1, 2)).fit(X[1])
messages_bow = bow_transformer.transform(X[1])
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

## Multinomial Naive Bayes Experimentation

In [35]:
a = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fit = [True, False]

parameters = {'alpha': a, 'fit_prior': fit}

In [ ]:
mnb = MultinomialNB()
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}
grid_search2 = GridSearchCV(mnb, parameters, cv=5, scoring=scoring, pre_dispatch= 8)
grid_search2.fit(messages_tfidf,y)

### Save the results

In [279]:
output1 = open('mnb_hre_29_08.pkl', 'wb')

pickle.dump(grid_search2.cv_results_, output1)

output1.close()

## Random Forest Experimentation

In [249]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 40)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [250]:
# Create the model
rf = RandomForestClassifier(random_state = 42)
 
# Define multiple metrics to be stored in the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, scoring=scoring, cv=5, pre_dispatch=8
                              , n_jobs=16)

# Fit the random search model
rf_random.fit(messages_tfidf,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'max_features': ['auto', 'sqrt']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=0)

### Save the results

In [24]:
output2 = open('dt_truth_same_sample_5fold.pkl', 'wb')

pickle.dump(rf_random.cv_results_, output2)

output2.close()

NameError: name 'rf_random' is not defined

In [4]:
# read python dict back from the file
pkl_file = open('results/10 fold/svm_rumor_5fold_user.pkl', 'rb')
mydict2 = pickle.load(pkl_file)
pkl_file.close()

## Decision Tree Experimentation

In [99]:
# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [103]:
dt = DecisionTreeClassifier(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}
dt_random = GridSearchCV(estimator=dt, param_grid=random_grid, cv=5, scoring= scoring, 
                                     pre_dispatch=8, refit='accuracy')

# Fit the random search model
dt_random.fit(messages_tfidf,y)

KeyboardInterrupt: 

### Save the results

In [283]:
output3 = open('dt_hre_29_08.pkl', 'wb')

pickle.dump(dt_random.cv_results_, output3)

output3.close()

## SVM Experimentation

In [254]:
Cs = [0.001, 0.01, 0.1, 1, 10]

gammas = [ 0.0001, 0.001, 0.01, 0.1, 1]

kernel = ['rbf', 'linear']

param_grid = {'C': Cs, 'gamma' : gammas, 'kernel': kernel}

In [255]:
svm = SVC(random_state = 42)
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}
grid_search = GridSearchCV(svm, param_grid, scoring=scoring, cv=5, pre_dispatch=8)
grid_search.fit(messages_tfidf,y)
grid_search.best_params_

{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}

### Save the results

In [285]:
output4 = open('svm_hre_29_08.pkl', 'wb')

pickle.dump(grid_search.cv_results_, output4)

output4.close()